In [5]:
import anndata
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

In [4]:
liger = importr('rliger')

In [7]:
# Load AnnData objects
raw_data = anndata.read_h5ad("2023_06_Guan_Chemical-induced_epigenome_resetting_for_regeneration/data/adata.h5ad.gz")
print(f"Data loaded (Raw): {raw_data.shape} cells x genes")
HVG_data = anndata.read_h5ad("2023_06_Guan_Chemical-induced_epigenome_resetting_for_regeneration//data/HVG_data.h5ad.gz")
print(f"Data loaded (Filtered + HVG): {HVG_data.shape} cells x genes")
pcHVG_data = anndata.read_h5ad("2023_06_Guan_Chemical-induced_epigenome_resetting_for_regeneration//data/pcHVG_data.h5ad.gz")
print(f"Data loaded (Filtered + HVG + ZScore): {pcHVG_data.shape} cells x genes")

Data loaded (Raw): (10328, 32738) cells x genes
Data loaded (Filtered + HVG): (10101, 919) cells x genes
Data loaded (Filtered + HVG + ZScore): (10101, 919) cells x genes


In [25]:
# HACERLO PARA CADA EXPERIMENTO
# Convertir el objeto AnnData a un DataFrame de pandas
somatic_data = HVG_data[HVG_data.obs.cell_state=="somatic",:].to_df()
# Activar la conversión automática entre pandas y R
pandas2ri.activate()
# Convertir el DataFrame de pandas a un DataFrame de R
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data_somatic = ro.conversion.py2rpy(somatic_data)
# Crear un objeto liger en R
ro.globalenv['r_data_somatic'] = r_data_somatic
ro.r('r_data_sparse_somatic <- as(as.matrix(r_data_somatic), "dgCMatrix")')

# Convertir el objeto AnnData a un DataFrame de pandas
reprogramming_data = HVG_data[HVG_data.obs.cell_state=="intermediate plastic state with a regeneration-like program", :].to_df()
# Activar la conversión automática entre pandas y R
pandas2ri.activate()
# Convertir el DataFrame de pandas a un DataFrame de R
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data_reprogramming = ro.conversion.py2rpy(reprogramming_data)
# Crear un objeto liger en R
ro.globalenv['r_data_reprogramming'] = r_data_reprogramming
ro.r('r_data_sparse_reprogramming <- as(as.matrix(r_data_reprogramming), "dgCMatrix")')


In [31]:
# Crear un objeto liger en R con la matriz esparsa
ro.r('liger_object <- createLiger(list(batch1 = r_data_sparse_somatic, batch2 = r_data_sparse_reprogramming))')
# Normalización y escalado de datos
ro.r('liger_object <- normalize(liger_object)')
# Seleccionar características variables -> he bajado el umbral + combine porque no seleccionaba ningun gen -> por que?
ro.r('liger_object <- selectGenes(liger_object, combine = "union", thresh = 0.1)')
ro.r('liger_object <- scaleNotCenter(liger_object)')

# Realizar la integración con liger
ro.r('liger_object <- optimizeALS(liger_object, k = 20)')
ro.r('liger_object <- quantileAlignSNF(liger_object)')

# Obtener los resultados de la integración
ro.r('integrated_matrix <- liger_object@H.norm')

# Realizar el clustering de Louvain
ro.r('liger_object <- clusterLouvainJaccard(liger_object, resolution = 1)')

# Realizar UMAP para la visualización
ro.r('liger_object <- runUMAP(liger_object, distance = "cosine", n_neighbors = 30, min_dist = 0.3)')

# Obtener los resultados de UMAP y clustering
umap_results = ro.r('liger_object@umap.coords')
clusters = ro.r('liger_object@clusters')
# Convertir la matriz integrada de R a pandas
integrated_matrix = ro.conversion.rpy2py(ro.r('integrated_matrix'))

ℹ calculating QC for dataset "batch1"


✔ calculating QC for dataset "batch1" ... done

ℹ calculating QC for dataset "batch2"
✔ calculating QC for dataset "batch2" ... done

ℹ Removing missing in dataset: "batch1"
ℹ Normalizing datasets "batch1"
ℹ Normalizing datasets "batch2"
✔ Normalizing datasets "batch2" ... done

✔ Normalizing datasets "batch1" ... done

ℹ Selecting variable features for dataset "batch1"
✔ ... 0 features selected out of 0 shared features.
ℹ Selecting variable features for dataset "batch2"
✔ ... 0 features selected out of 0 shared features.
✖ No genes were selected. Lower `thresh` values or set `combine = 'union'`
Error in `scaleNotCenter()`:
! No variable feature specified. Run `selectGenes()` first.
Run `rlang::last_trace()` to see where the error occurred.


RRuntimeError: Error in scaleNotCenter(liger_object) : 


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convertir resultados de R a DataFrame de pandas
umap_df = pd.DataFrame(umap_results, columns=['UMAP1', 'UMAP2'])
umap_df['Cluster'] = clusters

# Visualización con seaborn
plt.figure(figsize=(10, 8))
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='Cluster', palette='tab10', s=100)
plt.title('UMAP visualization of Louvain Clusters')
plt.show()

In [1]:
# Cargar los datos de scRNA-seq en una matriz AnnData
adata = ad.read_h5ad('2023_06_Guan_Chemical-induced_epigenome_resetting_for_regeneration/data/adata.h5ad.gz')

# Convertir el objeto AnnData a un DataFrame de pandas
data = adata.to_df()

# Activar la conversión automática entre pandas y R
pandas2ri.activate()

# Importar liger en R
liger = importr('liger')

# Convertir el DataFrame de pandas a un DataFrame de R
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)

# Crear un objeto liger en R
ro.globalenv['r_data'] = r_data
ro.r('liger_object <- createLiger(list(batch1 = r_data))')

# Normalización y escalado de datos
ro.r('liger_object <- normalize(liger_object)')
ro.r('liger_object <- scaleNotCenter(liger_object)')

# Realizar la integración con liger
ro.r('liger_object <- optimizeALS(liger_object, k = 20)')
ro.r('liger_object <- quantileAlignSNF(liger_object)')

# Obtener los resultados de la integración
ro.r('integrated_matrix <- liger_object@H.norm')

# Convertir la matriz integrada de R a pandas
integrated_matrix = ro.conversion.rpy2py(ro.r('integrated_matrix'))

# Crear un nuevo objeto AnnData con la matriz integrada
adata_integrated = ad.AnnData(X=integrated_matrix)

# Guardar el objeto AnnData integrado
adata_integrated.write('integrated_anndata.h5ad')

print("Integración con LIGER completada y guardada en 'integrated_anndata.h5ad'")


ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: import anndata
           ^


In [ ]:
# Instalar paquetes
# install.packages('BiocManager')
# BiocManager::install('ronammar/liger', dependencies=TRUE)

# Convertir el objeto AnnData a un DataFrame de pandas
df_data = HVG_data.to_df()

# Activar la conversión automática entre pandas y R
pandas2ri.activate()

# Importar liger en R
liger = importr('liger')

# Convertir el DataFrame de pandas a un DataFrame de R
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(df_data)

# Crear un objeto liger en R
ro.globalenv['r_data'] = r_data
ro.r('liger_object <- createLiger(list(batch1 = r_data))')

# Normalización y escalado de datos
ro.r('liger_object <- normalize(liger_object)')
ro.r('liger_object <- scaleNotCenter(liger_object)')

# Realizar la integración con liger
ro.r('liger_object <- optimizeALS(liger_object, k = 20)')
ro.r('liger_object <- quantileAlignSNF(liger_object)')

# Obtener los resultados de la integración
ro.r('integrated_matrix <- liger_object@H.norm')

# Convertir la matriz integrada de R a pandas
integrated_matrix = ro.conversion.rpy2py(ro.r('integrated_matrix'))

# Crear un nuevo objeto AnnData con la matriz integrada
adata_integrated = anndata.AnnData(X=integrated_matrix)

# Guardar el objeto AnnData integrado
adata_integrated.write('integrated_anndata.h5ad')

print("Integración con LIGER completada y guardada en 'integrated_anndata.h5ad'")

